# Check EMA Compliance

In [1]:
# import libraries
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")
week_ago = week_ago.strftime("%Y-%m-%d")

datapath = f"/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/export_tiki_{today}"
filepath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"



In [2]:
# load data
session = pd.read_csv(datapath + "/questionnaireSession.csv")
quest = pd.read_csv(datapath + "/questionnaires.csv")
df_active = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{proj_sheet}/export?format=csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data//questionnaireSession.csv'

In [ ]:
# session data
session["user"] = session["user"].str[:4]
session.rename(columns = {"completedAt": "quest_complete", "createdAt": "quest_create", "expirationTimestamp": "quest_expir"}, inplace=True)
session["quest_create"] = (pd.to_datetime(session["quest_create"],unit='ms'))
session["quest_complete"] = (pd.to_datetime(session["quest_complete"],unit='ms'))
session["quest_expir"] = (pd.to_datetime(session["quest_expir"],unit='ms'))

In [ ]:
quest = quest[["id", "name", "description"]]
quest.rename(columns = {"id": "questionnaire", "description": "quest_text"}, inplace=True)
session = pd.merge(session, quest, on="questionnaire")

### Combine with EMA ID 

In [ ]:
df_sess = session[["user", "questionnaire", "sessionRun", "quest_create", "quest_complete", "study"]]
df_sess = df_sess.rename(columns = {"user":"customer"})

In [ ]:
df_sess1 = df_sess.loc[df_sess.study.isin([24,25])]
sess_count1 = df_sess1.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count1 = sess_count1.rename(columns = {"quest_complete":"nquest_EMA1"})

In [ ]:
df_sess2 = df_sess.loc[df_sess.study.isin([33,34])]
sess_count2 = df_sess2.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count2 = sess_count2.rename(columns = {"quest_complete":"nquest_EMA2"})

In [ ]:
df_active.rename(columns = {"Pseudonym": "customer", "EMA_ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]
df_active["customer"] = df_active.customer.str[:4]

In [ ]:
df_complete = pd.merge(df_active, sess_count1, on="customer")
df_complete = pd.merge(df_complete, sess_count2, on="customer", how="outer")
df_complete.to_csv(f"ema_compliance_{today}.csv")

In [ ]:
df_finish = df_complete[df_complete.status == "Abgeschlossen"]

In [ ]:
df_finish.nquest_EMA1.describe()

In [ ]:
df_finish_long = df_complete[df_complete.status.isin(["Post_Erhebung_1","Post_Erhebung_2", "Erhebung_2_aktiv", "Abgeschlossen"])]

In [ ]:
(df_finish_long.groupby("study_version")["nquest_EMA1"].mean()/112)*100

In [ ]:
df_finish

In [ ]:
# Assuming df_finish_long is your DataFrame
# Group by "study_version" and calculate the mean of "nquest_EMA1" as a percentage
grouped_data = (df_finish_long.groupby("study_version")["nquest_EMA1"].mean() / 112) * 100

# Reset the index to make "study_version" a column
grouped_data = grouped_data.reset_index()


# Create a barplot or pointplot
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
sns.barplot(data=grouped_data, x="study_version", y="nquest_EMA1", palette="Paired")

for index, row in grouped_data.iterrows():
    plt.text(index, row["nquest_EMA1"], f"{row['nquest_EMA1']:.2f}%", ha="center", va="bottom", fontsize=14)

# Show the plot
plt.xticks(fontsize=14)  # Increase the fontsize of x-axis tick labels
plt.yticks(fontsize=14)  # Increase the fontsize of y-axis tick labels

plt.ylabel('')
plt.xlabel('')

# Show the plot
#plt.savefig("ema_grouped.png", dpi=300, format='png', bbox_inches='tight')

In [ ]:
df_finish_long.groupby("study_version")["nquest_EMA2"].count()